In [ ]:
import os
import pandas
import nilearn
import numpy as np
import scipy.stats as stat
import nibabel as ni
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

In [ ]:
# Now lets use the tools we've learned to do a few demonstrations of actual operations you
# might perform on a neuroimage. We will three examples: 
# 1) Extract values from an atlas
# 2) Create a correlation matrix from a rsfMRI image, and extract networks from that image
# 3) Voxelwise statistical operationn

In [ ]:
## DEMONSTRATION 1: Extract values from an atlas ##
# Lets say you have some PET images (or whatever) but you want to know the mean value within
# the different ROIs of a given atlas. There are better atlases out there, but since everyone
# uses it, lets use the Desikan-Killainy atlas (also know has the Freesurfer atlas).

In [ ]:
# First, lets load our image data and our atlas data

cwd = os.getcwd()


img = ni.load(
        os.path.join(cwd,'stuff/nan_snorm_002-S-4229_18F-AV1451_2016-02-10_P4_I635352.nii.gz'))
# Get individual subject PET data 
dat = img.get_data()

# Get atlas
jnk = ni.load(os.path.join(cwd,'stuff/dkt_atlas_1mm.nii.gz'))
atlas = jnk.get_data()




In [ ]:
# Now lets get our labels
labels = pandas.read_csv(os.path.join(cwd,'stuff/dst_labels.csv'),header=None,
                         skipinitialspace=True)
labels.columns = ['label','roi']
labels.index = labels['label'].astype(int)

In [ ]:
labels.head()

In [ ]:
# So, what we want to accomplish here is, for each roi, find all the values that are within
# that ROI and average them. The ROIs are labeled by the atlas. Because the atlas and data are
# in the same space, we can use indices from the atlas to index the PET data

In [ ]:
# Lets look at the unique values of the atlas -- in other words, lets look at all the
# different labels.
np.unique(atlas)

In [ ]:
# Looks like there are a few extras (they are just part of the cerebellum so we dont care
# about them). But also, notice the values are floats. Lets convert the atlas to integers so it
# better matches our labels.
atlas = atlas.astype(int)
np.unique(atlas)

In [ ]:
# Great. Now, there are a few ways to do this. One nice way would be to be smart with our masks
# and indexing. This will use a For loop.
def extract_values_with_atlas(dat,atlas,labels):
    for lab in labels.index:
        labels.ix[lab,'sub1_values'] = np.nanmean(dat[atlas==lab])

# Let me explain the last line. 

# First, we are going to be updating the spreadsheet with the new values. By writing 
# labels.ix[lab,'sub1_values'] = x, we are saying change the value in the spreadsheet in 
# column 'sub1_values', at index lab, to x. The column sub1_values does not exist yet, so it 
# will be automatically created.

# Next, when we say dat[atlas==lab], we are taking all values within certain coordinates in dat 
# -- specifically,  where the value is equal to lab at that coordinate in the atlas. In other
# words, lets say for the first iteration, lab is 1, therefore referring to coordinates within
# the caudal anterior cingulate. We are therefore taking only values in dat that are labeled
# with a 1 in the atlas (the caudal anterior cingulate). This only works because the images are
# in the same space and are the same size

# Finally, once we have those values, we are averaging them. I chose to use np.nanmean, which
# works quickly and ignores NaNs. If I didn't do this, the sum would be averaged by the total
# number of voxels, including voxels with NaNs. This would be inaccurate.

In [ ]:
# First we'll time it to get a sense of its speed
%timeit extract_values_with_atlas(dat,atlas,labels)

In [ ]:
# Mm.. not bad. On my machine it took 3.4s for all 80 ROIs -- ~25 ROIs a second. And how well
# did it work?
labels.head()

In [ ]:
# Quite well it seems! But can we make it faster? As we discussed before, the absolute most 
# efficient way to work on matrices in Python is to work with vectorized data. I.e. do
# manipulations on an entire matrix at once, rather than iterating through each point on the
# matrix. To do this, you need to either be good at math or be really clever. Here is an
# an example -- I'll run it first and then explain how it works.

def extract_values_vec(dat,atlas,labels,slc = None,col_lbl='vec_values'):
    
    count = np.bincount(atlas.flat)
    tot = np.bincount(atlas.flat, weights=dat.flat)
    if slc == None:
        slc = [0,len(count)]
    labels.ix[:,col_lbl] = (tot/count)[slc[0]:slc[1]]



In [ ]:
%timeit extract_values_vec(dat,atlas,labels,[1,81])

In [ ]:
# Much better! On my machine, I got about a 70x speed up! We did means on 80 ROIs in half a 
# second! And are the values the same?
labels.head()

In [ ]:
# Okay so lets walk through exactly how this worked, because its a bit tricky. This method is
# not very intuitive, but it makes use of existing python functions. Usually when you can do 
# that, you'll get better performance! Also notice there are no for loops and, in fact, there 
# is no explicit iteration whatsoever.

In [ ]:
# Lets look at the first line. What is this numpy function "bincount" doing?
np.bincount?

In [ ]:
# Lets ignore the weight argument for now because its not used in the first line.

# bincount basically assess the number of unique values in your matrix, and tells you how many
# points in your matrix have that value. And it does so very quickly. 

# In our case, we apply it to our atlas, and the number of points in our matrix is actually the
# number of voxels. So we are basically figuring out how many voxels equal each unique value.
# Or in other words, the size (in voxels) of each label of the atlas!

# Why are we doing this? Well, we know to calculate the mean, we divide the sum by the n. We 
# will be doing that separately for each region in the atlas, so this step is actually
# collecting the "n" of each region! In other words, we're calculating the demoninator for each
# mean calculation

# Have a look
count = np.bincount(atlas.flat)
count

In [ ]:
# Interesting to note that ~80% of the atlas is made up of zeros (the first value in the array 
# above).

# If this is difficult to understand, let's visualize it in a different way. Each value in the
# atlas represents a region label. The array above, then, is the number of voxels in each 
# region

# So:
list(zip(labels.roi, count[1:80]))


In [ ]:
# Next, we use np.bincount again, but this time, we make use of the "weight" argument. This 
# means that, instead of adding 1 for each cell for each unique value, we add 1*weight. Here,
# we put dat (our image data) as the weight. That means that each time a cell is found to 
# "belong" to a unique value, instead of adding 1, we add 1 * the value of that same cell in 
# dat. So in other words, this is literally just a really clever (and fast!) way to sum all of 
# the values of dat, separately for each value in atlas. Its exactly what we want to do!

# The result is a 83x1 array representing the sum of values in dat for each unique value in
# atlas. So, we've already done away with the need to iterate.

tot = np.bincount(atlas.flat, weights=dat.flat)
tot

In [ ]:
# Now all we need to do is get the mean. Since we know the mean is sum / n, we just need to
# divide our variable tot by the variable count, since count has the total number of cells for
# each unique variable in the atlas. And since we don't want the first value (0) or the last
# few values (81-82), we slice the final average so as to eliminate those values from the final
# result
avg = (tot/count)[1:81]
print(avg)



In [ ]:
# This would be a great time to let you guys do some experiments in working with imaging data.
# You know how to utilize all of these data structures, you know many of the basic and built-in
# Python commands, now its time to put them to the test.

# Below we'll have some exercises where you will try to do just some very basic analyses of a
# neuroimage. After that, we'll walk through some neuroimaging-specific Python functions, and
# some more complicated analyses

In [ ]:
# One more thing I'll teach you now is how to save an image. Its very easy with nibabel. All
# you need is some image data and an affine that matches the data. Because we are not changing
# the shape of the image, we can use the same affine of the image we loaded in the first place:
aff = jnk.affine
aff

In [ ]:
# Now we save the image a a Nifti image:
filename = os.path.join(cwd,'new_image')  # Define filename
nimg = ni.Nifti1Image(dat,aff) # Create new image
nimg.to_filename(filename) # Save new image to filename
os.listdir() # List contents to see if the image was created

# Feel free to open the image with your favorite image browser to see if worked

In [ ]:
# Now lets get rid of that image because we don't need it
os.remove(filename+'.nii')

In [ ]:
# This would be a great time to let you guys do some experiments in working with imaging data.
# You know how to utilize all of these data structures, you know many of the basic and built-in
# Python commands, now its time to put them to the test.

# Below we'll have some exercises where you will try to do just some very basic analyses of a
# neuroimage. After that, in Lesson 5C, we'll walk through some neuroimaging-specific Python 
# functions, and some more complicated analyses

In [ ]:
###### EXERCISES ##########

# NOTE: This time around, I've added some prompts to help you figure out what steps need to be
# taken

## PART A
# Write an image thresholding Python function. This will take a path to an image, a 
# threshold value, and an output name, as inputs. The function will threshold the image by 
# setting all voxels that are less than the supplied threshold to 0. Finally, the function will 
# write a new image which has been "thresholded". Test your function on the existing PET data 
# and look at the image you created to see if it works!

## PART B
# Threshold the imaging data such that only the top 5% of voxel values remain. Save this image
# and view it to see where the highest PET values are in the brain.

# Find top 5% index

# Find top 5% value

# Threshold image

# Write image to file

## PART C
# You have a hypothesis that PET signal will be higher in the Putamen than in the cortex.
# Create two matrices -- one containing flattened voxel values from the putamen, and another
# containing flattened voxel values from the rest of the brain. Make sure you don't get
# voxels with labels of 0, because those are outside the brain! Print the means of the two
# matrices. Then, run a t-test between these vectors to test your hypothesis. 
# NOTE: There are two Putamen ROIs, left and right. You'll want to combine values from both

# Create putamen matrix

# Create matrix for the rest of the brain

# Print means

# Run t-test


## PART D
# Using your favorite automation technique, run a t-test between the left Inferior Temporal 
# and every brain region. Then, create an image where regions are set to 0 if the average PET 
# signal in that region is significantly lower than the average Putamen signal, and otherwise 
# set the region to 1. 
# NOTE: For the labels, all of the left ROIs appear first, and right ROIs appear
# second.


In [ ]:
#### ANSWERS BELOW ####
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

In [ ]:
### ANSWERS TO EXERCISES

## PART A
# Write an image thresholding Python function. This will take a path to an image, a 
# threshold value, and an output name, as inputs. 

def threshold_image(image, thresh, outname):
    '''This function will threshold an image by a given value and return the thresholded
    image. Image should be a path to an existing image. thresh should be a number. outname
    should be string representing the filename of the new image to be created (without
    extension)'''

    # The function will threshold the image by setting all voxels that are less than the 
    # supplied threshold to 0. 
    jnk = ni.load(image)
    i_data = jnk.get_data()
    aff = jnk.affine
    
    i_data[i_data<thresh] = 0
    
    # Finally, the function will write a new image which has been thresholded. 
    nimg = ni.Nifti1Image(i_data,aff)
    nimg.to_filename(outname)
    
# Test your function on the existing PET data and look at the image you created to see if it 
# works!
file_in = os.path.join(cwd,
                       'stuff/nan_snorm_002-S-4229_18F-AV1451_2016-02-10_P4_I635352.nii.gz')
threshold_image(file_in, 0.8, 'thresh_image')

In [ ]:
## PART B
# Threshold the imaging data such that only the top 5% of voxel values remain. Save this image
# and view it to see where the highest PET values are in the brain.

# Find top 5% index:
flat_dat = dat.flatten()
img_size = len(flat_dat)
ind_5p = round(img_size * 0.95)

# Find top 5% value:
flat_dat.sort()
thr_val = flat_dat[ind_5p]
thr_val

# Threshold
tdat = deepcopy(dat) 
tdat[tdat<thr_val] = 0

# Write new image
nimg = ni.Nifti1Image(tdat,aff)
nimg.to_filename(os.path.join(cwd,'ex2_image'))

In [ ]:
## PART C
# You have a hypothesis that PET signal will be higher in the Putamen than in the cortex.


# Create two matrices -- one containing flattened voxel values from the putamen,
put_idx = [i for i in labels.index if labels.ix[i,'roi'] == 'Putamen']
put_mat = dat[(atlas==put_idx[0]) | (atlas==put_idx[1])]

# and another containing flattened voxel values from your the rest of the brain. 
n_atlas = deepcopy(atlas)
n_atlas[n_atlas == put_idx[0]] = 0
n_atlas[n_atlas == put_idx[1]] = 0
ctx_mat = dat[n_atlas > 0]

# Print the means of the two matrices
print(np.mean(put_mat))
print(np.mean(ctx_mat))

# Run a t-test between these vectors to test your hypothesis.
stat.ttest_ind(put_mat,ctx_mat)



In [ ]:
## PART D
# Using your favorite automation technique, run a t-test between the Inferior Temporal 
# and every brain region. Then, create an image where regions are set to 0 if the average PET 
# signal in that region is significantly lower than the average Putamen signal, and otherwise 
# set the region to 1. 

# There are many, many ways to do this. This is just one approach.

print('initializing')
it_idx = [i for i in labels.index if labels.ix[i,'roi'] == 'Inferior temporal'][0]
it_mat = dat[atlas==it_idx]

# Find which ROIs are significantly different
print('finding significant ROIs')
sig_indx = [i for i in labels.index if stat.ttest_ind(
                                                dat[atlas==i],
                                                it_mat)[1] < (0.05/80)]


# Of those, see which ROIs have lower average SUVRs using the spreadsheet we created earlier
print('refining...')
sig_indx = [i for i in sig_indx if labels.ix[i,
                                               'sub1_values'] < labels.ix[it_idx,
                                                                          'sub1_values']]
# Create the new image by binarizing the regions
print('binarizing')
thr_dat = deepcopy(dat)
n_atlas = deepcopy(atlas)


# And binarize...
for ind in sig_indx:
    n_atlas[n_atlas == ind] = 0
thr_dat[n_atlas == 0] = 0
thr_dat[n_atlas > 0] = 1

# Write image to file
nimg = ni.Nifti1Image(thr_dat,aff)
nimg.to_filename('ex4_image')


In [ ]:
# Clean up

i_2_del = ['ex2_image.nii','ex4_image.nii','thresh_image.nii']
for i in i_2_del:
    try:
        os.remove(i)
    except:
        continue
os.listdir()